Official Instagram API Resources

*   https://developers.facebook.com/docs/instagram/oembed


**The code snippets below (with a valid developer account) will provide all post IDs for a given user.**
```
# You can make a GET request to graph.facebook.com/zara/media
# and include an access token, but that would require approval on a developer account

import http.client

conn = http.client.HTTPSConnection("graph.facebook.com")
payload = ''
headers = {}
conn.request("GET", "/zara/media", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))


```
# or
```

import requests

url = "graph.facebook.com/zara/media"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)
```


Blockers

1.   Officially speaking, your account needs to be approved for developer access
2.   For a generic 3rd-party scraper, a personal account is needed.






# Task A: 

Scrape Instagram.py to fetch ~200 posts from the Zara Instagram page. This script fetches (i) image URLs, (ii) post caption (the text description of a post), and (iii) # likes. 

In [ ]:
pip install instaloader

     |████████████████████████████████| 54 kB 1.6 MB/s 
  Created wheel for instaloader: filename=instaloader-4.8.1-py3-none-any.whl size=58722 sha256=5d676635b9cc59f9f93c7cf13e093e7dff7e774dbbe76a0151f3abd227e094de
  Stored in directory: /root/.cache/pip/wheels/67/cc/31/c4f2a966180c23b0f66985bb9a90c389d447bc1a5ddffd35c9
Successfully built instaloader


In [ ]:
# Professor's way of scraping:

import instaloader, time
import pandas as pd
import sys
sys.setrecursionlimit(10000)
from datetime import datetime
from itertools import dropwhile, takewhile


In [ ]:
# Initialize
# Username: groupc_unstructure
# Password: Cgroup1234!
loader = instaloader.Instaloader()
loader.login("groupc_unstructure", "Cgroup1234!")

In [ ]:
# Create a Table
instaData = pd.DataFrame()

i=0
for post in instaloader.Profile.from_username(loader.context, 'zara').get_posts():
  instaData = instaData.append({'Caption': post.caption, 'Likes': post.likes, 'URL': post.url}, ignore_index = True)
  instaData.to_csv('zara.csv',index=False)
  i += 1
  if i > 200:
    break
print('Written to zara.csv file')

Written to zara.csv file


In [ ]:
# View the scraped data:
instaData.head()

,Caption,Likes,URL
0,Zara SRPLS Fall ‘21. Now available #zarasrpls,4482.0,https://instagram.fcjb7-1.fna.fbcdn.net/v/t51....
1,Zara SRPLS Fall ‘21. Now available #zarasrpls,8536.0,https://instagram.fcjb7-1.fna.fbcdn.net/v/t51....
2,Zara SRPLS Fall ‘21. Now available #zarasrpls,14835.0,https://instagram.fcjb7-1.fna.fbcdn.net/v/t51....
3,ZARA SRPLS. Collection 07. DRP01 COMING SOON #...,16143.0,https://instagram.fcjb7-1.fna.fbcdn.net/v/t51....
4,ZARA SRPLS. Collection 07. DRP01 COMING SOON #...,14484.0,https://instagram.fcjb7-1.fna.fbcdn.net/v/t51....


# Task B: 
Using the image URLs, obtain image labels (text) from Google Vision (cloud service)

In [ ]:
!pip install --user google-cloud

In [ ]:
!pip install --user google-cloud-vision

In [ ]:
!pip install --upgrade google-cloud-vision

In [ ]:
import xlrd
import os
import pandas as pd
#Important!!!
#After install google vision restart runtime
from google.cloud import vision

In [ ]:
#Google Vision API Credential
Application_Credentials = # redacted
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = Application_Credentials
client = vision.ImageAnnotatorClient()
image = vision.Image()

In [ ]:
# For csv read image_url.csv
file = pd.read_csv('image_url.csv')

In [ ]:
#Create a copy df
df_2 =file.copy()

In [ ]:
#Initialize labels column
df_2['labels'] = ''

In [ ]:
# Loop through URL send to Google Vision for image Analysis and get labels for each image
counter=0
for row in file['URL']:
  image_source = row
  image.source.image_uri = image_source
  response = client.label_detection(image=image)
  labels = response.label_annotations
  label_l = []
  for label in labels:
    label_l.append(label.description)
  s = ' '.join(label_l)
  df_2.at[counter,'labels'] = s
  counter = counter+1

In [ ]:
df_2

,URL,labels
0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Clothing Hair Face Skin Head Lip Chin Outerwea...
1,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Leg Sleeve Eyewear Standing Gesture Street fas...
2,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Head Outerwear Shoe Shoulder Leg Luggage and b...
3,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Outerwear Fur clothing Neck Sleeve Parka Stree...
4,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Neck Sleeve Grey Waist Font Denim Pattern Elec...
...,...,...
196,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Liquid Product Cosmetics Material property Fon...
197,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Nose Skin Head Lip Lipstick Chin Shoulder Eyeb...
198,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Eyebrow Eyelash Mouth Jaw Makeover Iris Black ...
199,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Water Cap Blue Azure Headgear Line Art Tints a...


In [ ]:
df_2.to_csv('image_url_label.csv')

# Task C: 
Create a column called binary (lowercase only) where value =1 (stands for high engagement) or 0 (stands for low engagement) based on whether the number of likes is above or below the median value. 


In [ ]:
import pandas as pd
# Read files from previous steps
fileA = pd.read_csv(r'/content/image_url_label.csv')
fileB = pd.read_csv(r'/content/zara.csv')

In [ ]:
# Basically merging the necessary information from both files
# Add the labels to the original file
file = fileB.copy()
file[['URL', 'labels']] = fileA[['URL', 'labels']]
file.head()

,Caption,Likes,URL,labels
0,Zara SRPLS Fall ‘21. Now available #zarasrpls,5398.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Clothing Hair Face Skin Head Lip Chin Outerwea...
1,Zara SRPLS Fall ‘21. Now available #zarasrpls,9006.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Leg Sleeve Eyewear Standing Gesture Street fas...
2,Zara SRPLS Fall ‘21. Now available #zarasrpls,15355.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Head Outerwear Shoe Shoulder Leg Luggage and b...
3,ZARA SRPLS. Collection 07. DRP01 COMING SOON #...,16149.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Outerwear Fur clothing Neck Sleeve Parka Stree...
4,ZARA SRPLS. Collection 07. DRP01 COMING SOON #...,14497.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Neck Sleeve Grey Waist Font Denim Pattern Elec...


In [ ]:
# Get the median
median = file.Likes.median()
median

41470.0

In [ ]:
# Create a new column and use True or False if the Like count is greater than the median (True if greater than, otherwise false)
file['binary'] = file['Likes'] >= median
# Change from True/False to 1/0
file['binary'] = file['binary'].astype(int)

In [ ]:
# Save the file
file.to_csv('with_binary.csv')
file

,Caption,Likes,URL,labels,binary
0,Zara SRPLS Fall ‘21. Now available #zarasrpls,5398.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Clothing Hair Face Skin Head Lip Chin Outerwea...,0
1,Zara SRPLS Fall ‘21. Now available #zarasrpls,9006.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Leg Sleeve Eyewear Standing Gesture Street fas...,0
2,Zara SRPLS Fall ‘21. Now available #zarasrpls,15355.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Head Outerwear Shoe Shoulder Leg Luggage and b...,0
3,ZARA SRPLS. Collection 07. DRP01 COMING SOON #...,16149.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Outerwear Fur clothing Neck Sleeve Parka Stree...,0
4,ZARA SRPLS. Collection 07. DRP01 COMING SOON #...,14497.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Neck Sleeve Grey Waist Font Denim Pattern Elec...,0
...,...,...,...,...,...
196,THE BEAUTY COLLECTION. \nNow available \nStill...,22503.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Liquid Product Cosmetics Material property Fon...,0
197,THE BEAUTY COLLECTION. \nON THE LINE \nCreativ...,39419.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Nose Skin Head Lip Lipstick Chin Shoulder Eyeb...,0
198,THE BEAUTY COLLECTION \nCHROMA \nAbstraction. ...,50777.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Eyebrow Eyelash Mouth Jaw Makeover Iris Black ...,1
199,THE BEAUTY COLLECTION \nCHROMA \nAbstraction. ...,30552.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Water Cap Blue Azure Headgear Line Art Tints a...,0


# Task D: 

Run a logistic regression with binary as the dependent variable, and the image_labels as independent variables. Before running the regression, replace the column label image_labels with text, since the script expects text to be the name for the column containing text. What is the accuracy (show the confusion matrix) of this prediction model? The idea is to be able to predict the engagement level for an image.

Accuracy = 1 - # prediction errors / total # cases

What accuracy do you get by using the post_caption words as the independent variables instead of image_labels? As in the first regression, change post_caption to text before running the logistic regression. Finally, what accuracy do you get by combining (concatenating) the image_labels and post_caption and using them together as independent variables? What can you conclude from your analysis? 

In [ ]:
# Reloading the << with_binary >> dataset
import pandas as pd
df_binary = pd.read_csv(r'with_binary.csv')

In [ ]:
ex = df_binary['labels'][3]
ex

'Outerwear Fur clothing Neck Sleeve Parka Street fashion Wig Fashion design Toy Liver'

In [ ]:
# Replacing special characters
import re
ex_words = re.findall('[A-Z][^A-Z]*', ex)
ex_words

['Outerwear ',
 'Fur clothing ',
 'Neck ',
 'Sleeve ',
 'Parka ',
 'Street fashion ',
 'Wig ',
 'Fashion design ',
 'Toy ',
 'Liver']

In [ ]:
# Taking out the ' ' blank spaces in the end of each word
ex_words[5].rstrip()

'Street fashion'

Creating the canonical vector for words in the **labels** column

In [ ]:
# Creating a canonical list of words in the column called << labels >>
# Which means: unique only words
labels_canonical = []

for rows in df_binary['labels']:
  for words in re.findall('[A-Z][^A-Z]*', rows):
    if words.rstrip() not in labels_canonical:
      labels_canonical.append(words.rstrip())

In [ ]:
# One hot encoding the each row based on the canonical list of words
# 0 if the word does not exist / 1 if yes
labels_encoded = []

for rows in df_binary['labels']:
  current_vector = []
  for canonical in labels_canonical:
    if canonical in words:
      current_vector.append(1)
    else:
      current_vector.append(0)
  labels_encoded.append(current_vector)

In [ ]:
# Loading and running a basic Logistic Regression model
# from Scikit Learn library
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Loading X << features >> data
X = pd.DataFrame(data=labels_encoded, columns=labels_canonical)

# Loading y << label / target >> data
y = df_binary['binary']

# Spliting training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Fit and predict model
clf = LogisticRegression(random_state=42).fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.score(X_test, y_test)

0.44776119402985076

In [ ]:
# Checking confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[30,  0],
       [37,  0]])

Creating the canonical vector for words in the **caption** column

In [ ]:
# Creating a canonical list of words in the column called << captions >>
# Which means: unique only words
captions_canonical = []

for rows in df_binary['Caption']:
  for words in {tag.strip("#") for tag in rows.split() if tag.startswith("#")}:
    if words.rstrip() not in captions_canonical:
      captions_canonical.append(words.rstrip())

In [ ]:
# One hot encoding again but on the captions columns
captions_encoded = []

for rows in df_binary['Caption']:
  current_vector = []
  for canonical in captions_canonical:
    if canonical in words:
      current_vector.append(1)
    else:
      current_vector.append(0)
  captions_encoded.append(current_vector)

In [ ]:
# Same process to fit a logistic regression
X = pd.DataFrame(data=captions_encoded, columns=captions_canonical)
y = df_binary['binary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = LogisticRegression(random_state=42).fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.score(X_test, y_test)

0.44776119402985076

Creating the canonical vector for words in the **captions and labels** columns combined

In [ ]:
# Concatenating both encoded columns
df_labels = pd.DataFrame(labels_encoded, columns=labels_canonical)
df_captions = pd.DataFrame(captions_encoded, columns=captions_canonical)
df_combined = pd.concat([df_labels, df_captions], axis=1)
df_combined

,Clothing,Hair,Face,Skin,Head,Lip,Chin,Outerwear,Hairstyle,Smile,Leg,Sleeve,Eyewear,Standing,Gesture,Street fashion,Waist,Collar,Knee,Blazer,Shoe,Shoulder,Luggage and bags,Bag,Human body,Neck,Fur clothing,Parka,Wig,Fashion design,Toy,Liver,Grey,Font,Denim,Pattern,Electric blue,Chest,Woolen,Building,...,Automotive wheel system,Hawker,Market,Vegetable,Fruit,Cosmetics,Beige,Landscape,Aeolian landform,Concrete,Skin care,Paint brush,Makeover,Snout,zarasrpls,zarawoman,zarakids,KarlTempler,zarastudio,zaraman,stevenmeisel,karltempler,charlottexzara,zaraathleticz,zaranewin,zarapets,zaraorigins,comingsoon,kassleditionsxzara,zarajewellery,zara,newin,newin.,JOINLIFE,zaramini,aw21,accesories,zarabeauty,thereisnobeautyonlybeauties,zaraneauty
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
197,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
198,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
199,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Running the same model on the data
X = df_combined
y = df_binary['binary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = LogisticRegression(random_state=42).fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.score(X_test, y_test)

0.44776119402985076

# Task E: 
Perform topic modeling (LDA) on the original image_labels. Choose an appropriate number of topics. You may want to start with 4-5 topics, but adjust the number up or down depending on the word distributions you get. Decide on suitable names for each topic. 

Now sort the data from high to low number of likes (don’t use the binary column, use the actual number of likes), and consider the highest and the lowest quartiles of likes. 

What are the main differences in the average topic weights of images across the two quartiles (e.g., greater weight of some topics in the highest versus lowest quartiles)? Show the main results in a table.


---



#### Perform Topic Modeling 

We start by performing Topic Modeling

In [ ]:
#We start by installing LDA
!pip install lda

     |████████████████████████████████| 351 kB 5.2 MB/s 
     |████████████████████████████████| 99 kB 9.0 MB/s 


In [ ]:
#And making the relevant imports
import numpy as np
import lda
import pandas as pd

In [ ]:
# Load the data
all_data_df = pd.read_csv('with_binary_ID.csv') #This file must have ID
# Print head
all_data_df.head()

,ID,Caption,Likes,URL,labels,binary
0,0,Zara SRPLS Fall ‘21. Now available #zarasrpls,5398.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Clothing Hair Face Skin Head Lip Chin Outerwea...,0
1,1,Zara SRPLS Fall ‘21. Now available #zarasrpls,9006.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Leg Sleeve Eyewear Standing Gesture Street fas...,0
2,2,Zara SRPLS Fall ‘21. Now available #zarasrpls,15355.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Head Outerwear Shoe Shoulder Leg Luggage and b...,0
3,3,ZARA SRPLS. Collection 07. DRP01 COMING SOON #...,16149.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Outerwear Fur clothing Neck Sleeve Parka Stree...,0
4,4,ZARA SRPLS. Collection 07. DRP01 COMING SOON #...,14497.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Neck Sleeve Grey Waist Font Denim Pattern Elec...,0


In order to use the LDA library we have to prepare:
* Corpus (list of sentences with words)
* Vocabulary
* Titles or Names of the documents (each instagram post is a document)

In [ ]:
# Create corpus
labels_df = all_data_df['labels']
corpus = list(labels_df.to_numpy())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#Vectorize, but consider only words with frequency 3 or more
vectorizer = CountVectorizer(min_df=3)
corpus_vect = vectorizer.fit(corpus)
X = corpus_vect.transform(corpus)

In [ ]:
print("Shape of sparse matrix with corpus is:", X.shape)
print("Sample of the values is:\n", X[0:2])

Shape of sparse matrix with corpus is: (201, 160)
Sample of the values is:
   (0, 20)	1
  (0, 22)	1
  (0, 46)	1
  (0, 64)	1
  (0, 65)	1
  (0, 69)	1
  (0, 87)	1
  (0, 107)	1
  (0, 127)	1
  (0, 130)	1
  (1, 9)	1
  (1, 24)	1
  (1, 45)	1
  (1, 48)	1
  (1, 59)	1
  (1, 81)	1
  (1, 83)	1
  (1, 129)	1
  (1, 134)	1
  (1, 135)	1
  (1, 153)	1


In [ ]:
#Now we convert the dense matrix to a numpy array
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]])

In the above section, note that we are disregarding words (labels) with 1 or 2 apperances. This is because a word that only shows up once is very confusing for topic modeling, and is very hard to get to conclusions of the relevance of the usage of that word in a topic or the engagement generated.

Now, we prepare the vocabulary and the titles.

In [ ]:
#Vocabulary is already generated by the vectorizer
vocab = tuple(corpus_vect.vocabulary_.keys())

In [ ]:
#Titles are the ID's of our instagram posts
titles = tuple(all_data_df['ID'].to_numpy())

In [ ]:
print("Example of vocabulary:", vocab[0:5])

Example of vocabulary: ('clothing', 'hair', 'face', 'skin', 'head')


In [ ]:
print("Example of titles:", titles[0:5])

Example of titles: (0, 1, 2, 3, 4)


Now is time to perform Topic modeling, note that:
* This execution is with 3 topics
* We tried using 2,4,and 5 topics, but the results had more labels "confused" between topics, or topics with inconsistent labels. 

In [ ]:
model = lda.LDA(n_topics=3, n_iter=1500, random_state=1)
model.fit(X.toarray())  # Fit the model
topic_word = model.topic_word_ 

INFO:lda:n_documents: 201
INFO:lda:vocab_size: 160
INFO:lda:n_words: 2186
INFO:lda:n_topics: 3
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -15335
INFO:lda:<10> log likelihood: -11160
INFO:lda:<20> log likelihood: -10667
INFO:lda:<30> log likelihood: -10585
INFO:lda:<40> log likelihood: -10566
INFO:lda:<50> log likelihood: -10520
INFO:lda:<60> log likelihood: -10510
INFO:lda:<70> log likelihood: -10553
INFO:lda:<80> log likelihood: -10528
INFO:lda:<90> log likelihood: -10545
INFO:lda:<100> log likelihood: -10506
INFO:lda:<110> log likelihood: -10563
INFO:lda:<120> log likelihood: -10551
INFO:lda:<130> log likelihood: -10547
INFO:lda:<140> log likelihood: -10535
INFO:lda:<150> log likelihood: -10528
INFO:lda:<160> log likelihood: -10540
INFO:lda:<170> log likelihood: -10530
INFO:lda:<180> log likelihood: -10595
INFO:lda:<190> log likelihood: -10548
INFO:lda:<200> log likelihood: -10536
INFO:lda:<210> log likelihood: -10507
INFO:lda:<220> log likelihood: -10531
INFO:lda:<230> log l

#### Assign Topic names

We start by exploring the results of the LDA analysis

In [ ]:
# Print top topic words 
n_top_words = 12
for i, topic_dist in enumerate(topic_word):
     topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
     print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: cap fun landscape hand goggles toy event jeans facial neck boats tire
Topic 1: clothing elbow headgear shorts flash leg photograph face hairstyle happy nature arts
Topic 2: wheel plant overcoat eye flash elbow curl forehead organ shades hat blazer


This doesn't help to identify the probability of each word belongin to each topic, so we create a dataframe that looks more like the spreasheet we reviewed in class:



In [ ]:
df = pd.DataFrame(topic_word, columns = list(vocab))
df

,clothing,hair,face,skin,head,lip,chin,outerwear,hairstyle,smile,leg,sleeve,eyewear,standing,gesture,street,fashion,waist,collar,knee,blazer,shoe,shoulder,luggage,and,bags,bag,human,body,neck,fur,design,toy,grey,font,pattern,electric,blue,building,black,...,equipment,symmetry,midnight,thigh,joint,performing,trousers,hat,shorts,fun,leisure,azure,sunglasses,vision,care,goggles,natural,environment,travel,water,tree,car,grass,glasses,boats,boating,supplies,muscle,sportswear,mountain,magenta,hood,motor,landscape,purple,lipstick,cylinder,liquid,iris,pink
0,0.000012,0.018404,0.000012,0.004917,0.000012,0.000012,0.000012,0.003691,0.000012,0.011047,0.000012,0.000012,0.000012,0.019630,0.000012,0.006143,0.000012,0.000012,0.007369,0.000012,0.000012,0.000012,0.007369,0.000012,0.020856,0.008595,0.000012,0.000012,0.000012,0.028212,0.000012,0.000012,0.042925,0.000012,0.000012,0.002464,0.000012,0.000012,0.000012,0.000012,...,0.000012,0.000012,0.000012,0.020856,0.019630,0.009821,0.020856,0.000012,0.000012,0.078482,0.004917,0.000012,0.000012,0.003691,0.013499,0.042925,0.011047,0.003691,0.000012,0.000012,0.000012,0.003691,0.003691,0.009821,0.026986,0.000012,0.000012,0.003691,0.000012,0.000012,0.009821,0.000012,0.000012,0.049056,0.000012,0.007369,0.000012,0.006143,0.001238,0.000012
1,0.055979,0.000014,0.025949,0.005474,0.019124,0.009569,0.005474,0.000014,0.025949,0.000014,0.027314,0.004109,0.004109,0.000014,0.004109,0.000014,0.006839,0.000014,0.000014,0.000014,0.000014,0.004109,0.000014,0.005474,0.000014,0.000014,0.000014,0.004109,0.012299,0.010934,0.005474,0.004109,0.000014,0.004109,0.000014,0.002744,0.021854,0.016394,0.004109,0.008204,...,0.020489,0.004109,0.020489,0.000014,0.000014,0.000014,0.000014,0.000014,0.030044,0.000014,0.001379,0.004109,0.005474,0.000014,0.006839,0.000014,0.006839,0.000014,0.000014,0.005474,0.004109,0.000014,0.000014,0.000014,0.000014,0.020489,0.009569,0.000014,0.008204,0.009569,0.000014,0.013664,0.000014,0.000014,0.012299,0.000014,0.010934,0.021854,0.009569,0.016394
2,0.000016,0.007796,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.026471,0.000016,0.000016,0.000016,0.000016,0.014021,0.000016,0.000016,0.000016,0.018690,0.000016,0.023358,0.029583,0.000016,0.006240,0.000016,0.000016,0.000016,0.006240,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.006240,0.000016,0.000016,0.000016,0.000016,0.000016,...,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.023358,0.029583,0.000016,0.000016,0.023358,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.018690,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.000016,0.017134,0.000016,0.000016,0.000016,0.000016,0.026471,0.000016,0.000016


Now we sort by each topic to find a name for them.

In [ ]:
#Sort by topic 0
df.transpose().sort_values([0], ascending = False).head(15)

,0,1,2
cap,0.087065,0.000014,0.000016
fun,0.078482,0.000014,0.000016
landscape,0.049056,0.000014,0.000016
hand,0.042925,0.000014,0.000016
toy,0.042925,0.000014,0.000016
goggles,0.042925,0.000014,0.000016
event,0.030665,0.000014,0.020246
jeans,0.030665,0.015029,0.000016
facial,0.029438,0.000014,0.000016
neck,0.028212,0.010934,0.000016


We can name Topic 0 as **Adventure**

In [ ]:
#Sort by topic 1
df.transpose().sort_values([1], ascending = False).head(15)

,0,1,2
clothing,0.000012,0.055979,0.000016
elbow,0.019630,0.050519,0.038920
headgear,0.000012,0.031409,0.000016
shorts,0.000012,0.030044,0.000016
flash,0.024534,0.028679,0.043589
photograph,0.000012,0.027314,0.000016
leg,0.000012,0.027314,0.000016
face,0.000012,0.025949,0.000016
hairstyle,0.000012,0.025949,0.026471
happy,0.000012,0.024584,0.000016


We can name the Topic 1 as **Photoshoot**.

In [ ]:
#Sort by topic 2
df.transpose().sort_values([2], ascending = False).head(15)

,0,1,2
wheel,0.000012,0.000014,0.063819
plant,0.000012,0.000014,0.057594
overcoat,0.000012,0.000014,0.054482
eye,0.000012,0.000014,0.048257
flash,0.024534,0.028679,0.043589
curl,0.000012,0.000014,0.038920
elbow,0.019630,0.050519,0.038920
forehead,0.000012,0.000014,0.035808
organ,0.000012,0.000014,0.034251
shades,0.000012,0.000014,0.031139


We can name the Topic 2 as **clothes**. 

Note that the similarity between Topic 1 and Topic 2 is high, however, we felt that keeping Topic2 as a separate topic helps give Topic 1 a more consistent meaning even though the meaning for Topic 3 is somewhat inconsistent. For example wheel and plant may not fit into this category, but looking at the "median" of the words they are related more to clothes.

Therefore we have 3 topics

* Adventure
* Photoshoot
* Clothes

Now let's see how much each document is related to each of these topics:

In [ ]:
doc_topic = model.doc_topic_
print("Post ID \t Adventure \t Photoshoot \t Clothes")
for i in range(201):
    print(titles[i], "\t\t", np.round(doc_topic[i][0],9), "\t", np.round(doc_topic[i][1],9), "\t", np.round(doc_topic[i][2],9) )

Post ID 	 Adventure 	 Photoshoot 	 Clothes
0 		 0.009708738 	 0.009708738 	 0.980582524
1 		 0.982300885 	 0.008849558 	 0.008849558
2 		 0.759398496 	 0.157894737 	 0.082706767
3 		 0.980582524 	 0.009708738 	 0.009708738
4 		 0.373493976 	 0.614457831 	 0.012048193
5 		 0.015873016 	 0.968253968 	 0.015873016
6 		 0.983739837 	 0.008130081 	 0.008130081
7 		 0.274336283 	 0.008849558 	 0.716814159
8 		 0.834586466 	 0.157894737 	 0.007518797
9 		 0.539823009 	 0.008849558 	 0.451327434
10 		 0.008130081 	 0.008130081 	 0.983739837
11 		 0.097345133 	 0.008849558 	 0.89380531
12 		 0.174603175 	 0.80952381 	 0.015873016
13 		 0.012048193 	 0.855421687 	 0.13253012
14 		 0.312883436 	 0.680981595 	 0.006134969
15 		 0.11827957 	 0.870967742 	 0.010752688
16 		 0.983739837 	 0.008130081 	 0.008130081
17 		 0.008849558 	 0.539823009 	 0.451327434
18 		 0.008130081 	 0.983739837 	 0.008130081
19 		 0.97260274 	 0.01369863 	 0.01369863
20 		 0.383458647 	 0.609022556 	 0.007518797
21 		 0.

#### Sort data by likes (Find topics for top and low quartiles)

##### Top quartile (most liked)

In [ ]:
#Top 50 posts by likes
all_data_df.sort_values(['Likes'], ascending=False).head(50)

,ID,Caption,Likes,URL,labels,binary
87,87,New collection #zarawoman #zaranewin,232215.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Footwear Face Joint Hand Shoe Shoulder Arm Dre...,1
148,148,Waves of Summer #zarawoman,220388.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Water Sky People on beach Leg People in nature...,1
86,86,New collection APRÈS-VACANCES #zarawoman #zara...,185733.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Outerwear Shoulder White Dress shirt Street fa...,1
54,54,The new pet collection #zarapets,174780.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Outerwear Fur clothing Sleeve Gesture Headgear...,1
159,159,Going out tops. Low-slung skirts. Party like i...,167738.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Hair Arm Leg Muscle Human body Flash photograp...,1
156,156,Draped lingerie-style dress #zarawoman,158442.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Water Sky Flash photography People in nature W...,1
106,106,High rise full length printed jeans #zarawoman,126171.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Glasses Joint Lip Smile Vision care Shoulder F...,1
62,62,ZARA ORIGINS - A new line whose mission is to ...,121029.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Suit trousers Outerwear Vision care Sleeve Fla...,1
180,180,"©️ F JERRY, LLC 2021 PRINT T-SHIRT @dudewithsign",116514.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Hairstyle Photograph White Window Street fashi...,1
182,182,"©️ F JERRY, LLC 2021 PRINT T-SHIRT @dudewithsign",99000.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Footwear Jeans Wheel White Street fashion Leg ...,1


In [ ]:
#We cant get the post IDs for these
top_imgs = all_data_df.sort_values(['Likes'], ascending=False).head(50)
top_imgs['ID'].to_numpy()

array([ 87, 148,  86,  54, 159, 156, 106,  62, 180, 182, 130, 158, 157,
       188, 105, 167,  60, 134, 119, 143, 107,  70,  81, 114,  93, 161,
       147, 111, 140, 117,  56, 190, 136,  84, 104, 103, 192, 135,  76,
        97,  32, 186, 154, 153,  49,  40, 149,  82,  44,  34])

Now, we calculate the average topic representation in these top 50 results

In [ ]:
#Init sum values
sum_topic_0 = 0
sum_topic_1 = 0
sum_topic_2 = 0
sum_topic_3 = 0

#Loop over the IDs we found above
for id in top_imgs['ID']:
  #For each ID get the "belonging" to each topic
  sum_topic_0 += doc_topic[id][0]
  sum_topic_1 += doc_topic[id][1]
  sum_topic_2 += doc_topic[id][2]

print("Topic 0/Adventure:", sum_topic_0/50)
print("Topic 1/Photoshoot:", sum_topic_1/50)
print("Topic 2/Clothes:", sum_topic_2/50)

Topic 0/Adventure: 0.46571906568709187
Topic 1/Photoshoot: 0.27388732331179505
Topic 2/Clothes: 0.26039361100111325


We find that the Topic that is more represented in the posts with more likes is **Topic 0/Adventure**.

##### Bottom quartile (less liked)

In [ ]:
#Bottom 50 posts by likes
all_data_df.sort_values(['Likes'], ascending=True).head(50)

,ID,Caption,Likes,URL,labels,binary
33,33,Chapter 2. The investigation. Discover who is ...,5046.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Wood Grey Tints and shades Felidae Small to me...,0
0,0,Zara SRPLS Fall ‘21. Now available #zarasrpls,5398.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Clothing Hair Face Skin Head Lip Chin Outerwea...,0
1,1,Zara SRPLS Fall ‘21. Now available #zarasrpls,9006.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Leg Sleeve Eyewear Standing Gesture Street fas...,0
15,15,Zara Studio Fall ‘21 Collection\nArt Director:...,11716.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Sleeve Gesture Fixture Tints and shades Rectan...,0
20,20,Art Director: Fabien Baron @fabienbaron\nStyli...,12233.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Window Comfort Cloud Textile Sleeve Curtain Wo...,0
17,17,Art Director: Fabien Baron @fabienbaron\nFilm ...,12358.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Flash photography Jaw Beard Eyelash Facial hai...,0
5,5,ZARA SRPLS. Collection 07. DRP01 COMING SOON #...,12369.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Building Black-and-white World Font Tower bloc...,0
61,61,ZARA ORIGINS is a project focused upon buildin...,13553.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Light Azure Underwater Water Art Fin Font Elec...,0
25,25,Art Director: Fabien Baron @fabienbaron\nFilm ...,14316.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Footwear Shoe One-piece garment Leg Overcoat S...,0
4,4,ZARA SRPLS. Collection 07. DRP01 COMING SOON #...,14497.0,https://scontent-hkt1-2.cdninstagram.com/v/t51...,Neck Sleeve Grey Waist Font Denim Pattern Elec...,0


In [ ]:
#We cant get the post IDs for these
top_imgs = all_data_df.sort_values(['Likes'], ascending=True).head(50)
top_imgs['ID'].to_numpy()

array([ 33,   0,   1,  15,  20,  17,   5,  61,  25,   4,  13,   2,   3,
       118, 137,  45,  18,  47,  46, 142, 175, 176, 174, 121,  63,   8,
       196,  52,  41,   7,  85,  65,  51,  66,  53,  68,  71, 193, 179,
       144,  67, 195, 178, 183,  64, 112, 125,   6, 141, 185])

In [ ]:
#Init sum values
sum_topic_0 = 0
sum_topic_1 = 0
sum_topic_2 = 0
sum_topic_3 = 0

#Loop over the IDs we found above
for id in top_imgs['ID']:
  #For each ID get the "belonging" to each topic
  sum_topic_0 += doc_topic[id][0]
  sum_topic_1 += doc_topic[id][1]
  sum_topic_2 += doc_topic[id][2]

print("Topic 0/Adventure:", sum_topic_0/50)
print("Topic 1/Photoshoot:", sum_topic_1/50)
print("Topic 2/Clothes:", sum_topic_2/50)

Topic 0/Adventure: 0.2898367793061969
Topic 1/Photoshoot: 0.5119227698960684
Topic 2/Clothes: 0.19824045079773453


We see that the topic that is more represented in the posts that are less liked is **Topic 1/Photoshoot**.

# Task F: 
What advice would you give Zara if it wants to increase engagement on its Instagram page based on your findings?

Let's start by exploring a couple of pictures based on the findings above.

#### Pictures from Adventure (Topic 0)

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1u-HscYDyhFEYgKnqRARMGZDoJ34siYJb' width="500" height="400" />
<figcaption>ID 143 - LABELS: Sky Plant Mountain Cloud People in nature Happy Tree Natural landscape Grass Leisure</figcaption></center>
</figure>

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1C20AInyWKWxGENFIzAzIXDmCqW-WERvn' width="500" height="400" />
<figcaption>ID 194 - LABELS: Sky Beige Wood Tints and shades Facade Landscape Tree Art Aeolian landform Concrete</figcaption></center>
</figure>

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=13s_o0puE-c1yrVM_L_pNp_O3a9OD-gZ5' width="500" height="400" />
<figcaption>ID 146 - LABELS: Smile Human Flash photography Happy Gesture Interaction Leisure Fun Grass Friendship</figcaption></center>
</figure>




#### Pictures from Photoshoot (Topic 1)

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1Yj1VOwhkUzyfEkIbM8gStgGZ-SXoKI9R' width="500" height="500" />
<figcaption>ID 37 - LABELS: Jeans Outerwear Flash photography Sleeve Gesture Font T-shirt Rectangle Happy Elbow</figcaption></center>
</figure>

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1hcOJcr9JyYhGjBHuHaaZqwF9M7YsSWC8' width="400" height="500" />
<figcaption>ID 36 - LABELS: Clothing Face Hair Head Jeans Outerwear Shoe Leg Sleeve Waist</figcaption></center>
</figure>

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1N8Eh27cc-adSWiaMlTm5eAvo1nViHCYl' width="300" height="400" />
<figcaption>ID 80 - LABELS: Clothing Forehead Hair Glasses Jeans Chin Vision care Photograph Sunglasses Goggles</figcaption></center>
</figure>

The main differences between the "Adventure" topic that is more represented in the pictures with more likes, and the "Photoshoot" topic that is more represented in the pictures with less likes are:

The "Adventure" topic contains the labels landscape and fun, as if the people in the pictures have a context and an activity that they are doing within that context. Representative pictures of this topic always include a background (a.k.a landscape) that creates a setting in which some story may exist and drives the audience to ask questions about it, i.e. what is the large wall in the background? Why are these women together? Is it a party? Where are the snowy mountains located? Is the girl travelling?

Also, even if the label "fun" does not show up in all of the pictures in the sample above, the word fun is highly representative of this topic and it correlates with higher engagement.

Finally, we can see that the presence of other elements aside from the person creates a yuxtaposition between the clothes and the background that shift the focus from the person and clothes themselves. For example, the contrast between the urban style of the clothing, and the rural background creates interest in the contrast between the background and the fashion decision. 

On the other hand, the "Photoshoot" topic contains the labels clothing, photograph/photography, and flash. As if the people in the pictures are just there as a model in a photoshoot. Compared to the "Adventure" topic, these pictures do not evoke questions, and they lack the context and the dynamism of these previous pictures. We can see that even though there is not a camera in the frame, or a cameraman, the Google Vision API labeled them as photographs and that speaks to the clear attempt of posing to the camera. 

In this topic, it is clear that the clothes are the main focus of attention, and that seems to be discouraging engagement. Our hypothesis is that consumers of Zara's content do not want to feel as though they are being sold clothing when they look at these photographs, they seem to be willing to "buy" the experience that comes with wearing clothing in a given context, especially if that context contains some fun. Furthermore, it's possible that buyers of Zara are buying an identity characterized as fashionable, in vogue, or unconventional that demands attention.

Therefore, we would recommend Zara to include more picture with landscapes or some kind of background/setting. Zara can also further experiment with contexts where people are having fun. Importantly, Zara has reason to avoid pictures that are evidently photoshoots, and are not telling a story beyond a model wearing clothes.




#### Challenges with the topic modeling approach 

Even though, the results presented above are solid in terms of being backed by data, we believe that topic modeling alone was not able to extract all the relevant features that generate engagement within Zara's buyers. For example, we see that some pictures with low engagement are simply low in resolution and or color, which are characteristics of the photos that topic modeling will fail to capture. On the other hand, some photos have a big engagement simply because the photo has a high quality, or due to some unconventional feature like having a plus-sized model, again things that will not show up in topic modeling. 

Another challenge, is that we depend on the Google Cloud API accuracy and comprehensiveness. Sometimes, as humans, we can see "labels" in the picture that might be relevant but that the API was not able to capture and thus limiting the analysis to some extent. Empirically, we see that the API is returning different results based on the number of discrete objects that exist in a photo, certain features can be overweighted or underweighted. For example, "elbow" was considered to be a major representative word of Topic 1 (Photoshoot) but it may be only because of the lack of discrete features in the pictures. 